---
#**Ecuacion diferencial ordinaria de orden superior**

David Felipe Bambague (dbambague@unal.edu.co)

---

  Vamos a resolver la ecuación difrerncial que da cuenta de las oscilaciones de una viga con extremo fijo y extremo libre.

  \begin{equation*}
      EI\frac{d^4\eta}{dx^4}=0
  \end{equation*}

  con $E=1$, $F=1$ y $I=2$. Las condiciones de frontera del problema son:

  \begin{equation*}
      \eta(0)=0\\
      \eta^{\prime}(0)=0\\
      \eta^{\prime\prime}(L)=0\\
      \eta^{\prime\prime\prime}(l)=\frac{F}{EI}
  \end{equation*}

In [1]:
!pip install deepxde

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.2.1 -> 22.3
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
import deepxde as dde
import numpy as np
import matplotlib.pyplot as plt

#Usar doble precisión si la GPU lo soporta. Float32 por defecto.
dde.config.real.set_float64()

Using backend: tensorflow.compat.v1

2022-10-20 22:23:13.778365: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-10-20 22:23:13.781481: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-20 22:23:13.781492: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Instructions for updating:
non-resource variables are not supported in the long term


2022-10-20 22:23:14.914145: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-10-20 22:23:14.914190: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (pop-os): /proc/driver/nvidia/version does not exist


#1. Definir la Ecuación a Resolver

In [3]:
I=2.0
F=1.0
E=1.0
def ode(x, y):
    dy_dt = dde.grad.jacobian(y, x)
    d2y_dt2 = dde.grad.hessian(y, x)
    d3y_dt3 =dde.grad.jacobian(d2y_dt2,x)
    d4y_dt4 = dde.grad.jacobian(d3y_dt3, x)

    return E*I*d4y_dt4 

#**2. Definir Dominio de la Ecuación**

In [4]:
geom = dde.geometry.TimeDomain(0, 3)


#**3. Definir Condiciones Iniciales y de Frontera**

In [5]:
#condición para x=0
def in_boundary_1(x, on_initial):
    return on_initial and np.isclose(x[0], 0)
    
#condicion para x=L
def in_boundary_2(x, on_initial):
    return on_initial and np.isclose(x[0], 1)

In [6]:
def IC_1(inputs, outputs, x):
    return 0 
def IC_2(inputs, outputs, x):
    return dde.grad.jacobian(outputs, inputs)
def IC_3(inputs, outputs, x):
    return dde.grad.hessian(outputs, inputs)
def IC_4(inputs, outputs, x):
    inputs2 = dde.grad.jacobian(outputs, inputs)
    return dde.grad.hessian(outputs, inputs2)-0.5

In [7]:
Ic1 = dde.icbc.IC(geom, IC_1, in_boundary_1)
Ic2 = dde.icbc.OperatorBC(geom, IC_2, in_boundary_1)
Ic3 = dde.icbc.OperatorBC(geom, IC_3, in_boundary_2)
Ic4 = dde.icbc.OperatorBC(geom, IC_4, in_boundary_2)

#**4. Crear Datos de Entrenamiento**

In [8]:
data = dde.data.TimePDE(geom, ode, [Ic1,Ic2, Ic3, Ic4], 64, 4, solution=func, num_test=500)


NameError: name 'func' is not defined

#**5. Crear la Red Neuronal**

In [ ]:
layer_size = [1] + [30]*3 + [1]
activation = "tanh"
initializer = "Glorot uniform"
net = dde.nn.FNN(layer_size, activation, initializer)

#**6. Compilar y Entrenar el Modelo**

In [ ]:
# Modelo
model = dde.Model(data, net)

# Optimizador Adam
model.compile("adam", lr=0.001, loss_weights=[0.01, 1, 1], metrics=["l2 relative error"] ) 
losshistory, train_state = model.train(iterations=7000)